## MNIST(kaggleバージョン)をPytorchで実行する。
### やること
1. データの読み込み〜Dataset化
2. Dataloaderの定義
3. ネットワークの定義
4. 学習処理の定義
5. 学習結果の可視化(損失、正答率、ラベルごとの間違い数)
6. テスト処理の定義
7. テスト結果の可視化（正答率、ラベルごとの間違い数）

### 参考
- Dataset, DataLoaderについて：  
    - https://www.kaggle.com/pinocookie/pytorch-dataset-and-dataloader  
    - https://qiita.com/mathlive/items/2a512831878b8018db02
- 学習処理の記述方法について：
    - https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py


## ライブラリの読み込み

In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm

## モデルの定義

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() ## run the Parent Class's .__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3) # (in_channnel, out_channel, kernel_size)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(400, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Softmax()

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) ## J
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # x.shape = (8,400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net().double()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fc4): Softmax(dim=None)
)


## Datasetの定義
- ここでは、データを格納しているTensorを元にDatasetクラスを作成し、Pytorch上で扱いやすくする。
- TorchVision標準のデータセットを用いる場合、以下の流れとなるらしい。
    - transformsによる前処理の定義(torchvision)
        - transform：データの前処理全般を定義するもの。
        - 今回は各pixelの値を255.0で除算する前処理を入れる。
        - 画像なら(縦、横、チャネル)->(チャネル、縦、横)と変換する、ピクセル数を補正する、左右反転、回転させるなどの機能がある。
    - Datasetsによる前処理&ダウンロード
    - DataloaderによるDatasetの使用
- 参考1：https://qiita.com/mathlive/items/2a512831878b8018db02
- 参考2：書籍：つくりながら学ぶ！Pytorchによる発展ディープラーニング 小川雄太郎 著

In [4]:
class ImageTransform():
    """
    画像に対する前処理を行うクラス。以下の機能を持つ。
    pandas.DataFrameを入力としてTensorに変換する。
    その後、各pixelの値を0〜1にするため、255で除算する。
    なお、学習時と診断時での処理内容は同じであるとする。
    
    Attributes
    ----------
    None
    
    """
    def __init__(self):
        pass
    
    def __call__(self, df_pic):
        """
        学習時と診断時での前処理は同じものとする。
        本当は学習時のみノイズを加えたり色々する。
        """
        out_label = df_pic.label
        out_img = df_pic.iloc[:,1:].values.astype(np.uint8).reshape((1, 28, 28))
        out_label = torch.tensor(y_train.values)
        out_img = torch.tensor(x_train.values)
        out_img = out_img/255.0
        return out_img, out_label

trans = ImageTransform

## DataSetsの作成
- torch.utils.data.Datasetクラスを継承して作成する。
- メソッドには以下がある。
    - init:コンストラクタ、クラス定義時に実行され、値の初期化などを行う。
    - len:画像の枚数を返す。
    - getitem:Datasetからデータを1レコード分返す。

In [8]:
class MnistDatasets(data.Dataset):
    def __init__(self, input_filename, transform = None):
        self.data = pd.read_csv(input_filename)
        self.transform = trans
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_transformed = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((1, 28, 28))/255.0
        img_label = self.data.iloc[idx, 0]
        
        return img_transformed, img_label

ds = MnistDatasets("./01_input/train.csv")

## Dataloaderを作成する

In [11]:
dataloader = data.DataLoader(ds, batch_size=8, shuffle=False)

## 損失関数と最適化手法を定義する
損失関数には交差エントロピー損失を使用する。

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

## これまでに作成した関数を用いて学習を行なう。
- パラメータの初期化
- for epoch
    - for ミニバッチ
        - 順伝播
        - 損失の計算
        - パラメータ最適化
            - 損失の勾配の計算
            - パラメータの更新
            

In [16]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data_jj in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data_jj
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.double())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

/home/miso/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   500] loss: 1.499
[1,  1000] loss: 1.499
[1,  1500] loss: 1.494
[1,  2000] loss: 1.498
[1,  2500] loss: 1.491
[1,  3000] loss: 1.495
[1,  3500] loss: 1.498
[1,  4000] loss: 1.493
[1,  4500] loss: 1.493
[1,  5000] loss: 1.501
[2,   500] loss: 1.494
[2,  1000] loss: 1.499
[2,  1500] loss: 1.499
[2,  2000] loss: 1.496
[2,  2500] loss: 1.494
[2,  3000] loss: 1.495
[2,  3500] loss: 1.501
[2,  4000] loss: 1.486
[2,  4500] loss: 1.490
[2,  5000] loss: 1.496
Finished Training


## 学習済みモデルを用いた予測の実行
- メモリにデータが乗り切るよう、100件ごとに予測値を算出する。
- PCのメモリが足りず全データを一度に処理できなかったため。

In [ ]:
a = 0
for j in tqdm(np.arange(280)):
    x_test_tmp = x_test.iloc[100*j:100*j+100, :]
    x_transformed = x_test_tmp.values.astype(np.uint8).reshape((x_test_tmp.shape[0], 1, 28, 28))/255.0
    tensor_x = torch.Tensor(x_transformed).double()
    
    ## 予測の実行
    tmp_pred = net(tensor_x)
    tmp_pred = np.argmax(tmp_pred.detach().numpy().copy(), axis=1)
    if a==0:
        pred = tmp_pred
        a+=1
    else:
        pred = np.r_[pred, tmp_pred]

## あとはpd.DataFrame(pred).to_csv()して列名をつけてSubmitする。
## Pandasでのインデックス名の付け方を忘れた

## その他：Pythonにおけるクラスの概念
- "class hogehoge():" として定義する。引数には継承元の親クラスが入る。
- アンダースコア2本で囲まれるメソッドは特殊メソッドと呼ばれる。
    - init：インスタンスを生成する時点で実行される。
    - call：生成したインスタンスを()つきで実行すると実行される。
    - getitem:生成したインスタンスを[]つきで実行すると実行される。鍵カッコにはインデックスを示す整数が入る。
- self.super().｛メソッド名｝とすると親クラスのメソッドを利用できる。